# Setup Empty Pipeline with Screen


In [1]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors.basic import SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

# Create a screen for our Pipeline
mean_close_10 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=10
)

universe = mean_close_10 > 10

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen=universe)

# Setup Zipline Engine

In [2]:
import helper
from zipline.utils.calendars import get_calendar
from zipline.data import bundles

# Name of bundle
EOD_BUNDLE_NAME = 'quantopian-quandl'

# Load the data bundle
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Setup the engine to look at the top 500 stocks who have had the highest rolling Average Dollar Volume
# over a 120-day window -- This is arbitrary and we can use this parameter to refine which stocks we
# want in our universe
universe = mean_close_10.top(500) 

# Select the trading calendar that will be used as a reference when slicing the data
trading_calendar = get_calendar('NYSE') 

# Load the bundle we configured in the previous step into the engine
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Create the engine -- the details of this function are in the utils.py file
engine = helper.build_pipeline_engine(bundle_data, trading_calendar)

# Run Pipeline Engine

In [3]:
import pandas as pd

# Set the start and end dates
start_date = pd.Timestamp('2017-01-05', tz = 'utc')
end_date = pd.Timestamp('2017-01-05', tz = 'utc')

# Run our pipeline for the given start and end dates
pipeline_output = engine.run_pipeline(pipeline, start_date, end_date)

# Get Universe Tickers

In [4]:
# Get the values in index level 1 and save them to a list
universe_tickers = pipeline_output.index.get_level_values(1).values.tolist()

# Create Data Portal

In [5]:
from zipline.data.data_portal import DataPortal

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = bundle_data.equity_daily_bar_reader.first_trading_day,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

# View Historical Data

Get the OHLC + V data for a given time period. This data will be split into individual dataframes.

In [6]:
def get_field_data(data_portal, trading_calendar, assets, start_date, end_date, field):
    
    # Set the given start and end dates to Timestamps. The frequency string C is used to
    # indicate that a CustomBusinessDay DateOffset is used
    end_dt = pd.Timestamp(end_date, tz='UTC', freq='C')
    start_dt = pd.Timestamp(start_date, tz='UTC', freq='C')

    # Get the locations of the start and end dates
    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    # return the historical data for the given window
    return data_portal.get_history_window(assets=assets, end_dt=end_dt, bar_count=end_loc - start_loc,
                                          frequency='1d',
                                          field=field,
                                          data_frequency='daily')

# The window of data to obtain
start_date = '2012-01-05'
end_date = '2017-01-05'

# Get the open data
open_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                          start_date, end_date, 'open')

# Get the high data
high_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'high')

# Get the low data
low_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                         start_date, end_date, 'low')

# Get the closing data
close_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'close') 

# Get the volume data
volume_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                            start_date, end_date, 'volume')

from pypf.instrument import DataframeInstrument
from pypf.chart import PFChart

def generate_pf_chart(ticker, historical_dfs, duration):
    ''' 
        This function will create a p&f chart for the given ticker using historical data
        
        @param ticker: (str) ticker of asset to create P&F chart for
        @param historical_dfs: (pd.DataFrame) DataFrame holding historical ticker data
        
        return: PFChart object from which P&F chart/data can be extracted
    
    '''
    # Set up dataframe instrument
    try:
        df = historical_dfs[ticker][0]
    except:
        raise ValueError('Ticker passed does not exist in historical dataset')

    # Format date and volume values
    df['Date'] = df.index.astype(str)
    df['Date'] = df['Date'].str.slice(0,10)
    df['Volume'] = df['Volume'].astype(int)

    # Tes
    dfi = DataframeInstrument(ticker, dataframe=df)

    # Create pf chart
    chart = PFChart(dfi, duration=duration)
    chart.create_chart()
    return chart

# Combine OHLC dataframes into singular dataframe
Here we combine the four individual dataframes representing OHLC + V data into one historical dataframe.

In [7]:
import numpy as np
from helper import beautify_tickers

def get_hist_data(universe_tickers):
    #Create dataframe and append blank row
    historical_dfs = pd.DataFrame(columns=universe_tickers)
    historical_dfs = historical_dfs.append(pd.Series([np.nan]), ignore_index=True)

    # Loop through each universe ticker and create a combined dataframe for that ticker
    for ticker in universe_tickers:
        # Get individual series representing the OHLCV data
        open_series = open_data[ticker]
        high_series = high_data[ticker]
        low_series = low_data[ticker]
        close_series = close_data[ticker]
        volume_series = volume_data[ticker]

        # Combine these series into 1 dataframe
        columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        df = pd.concat([open_series,high_series, low_series, close_series, volume_series], axis=1)
        df.columns = columns    

        # Save this dataframe to historical_dfs
        historical_dfs[ticker] = pd.Series([df])


    # Change the columns to be more human readable
    columns = beautify_tickers(universe_tickers)
    historical_dfs.columns = columns
    return historical_dfs

# Format P&F Chart Data For Analysis
Create a new DataFrame to hold the P&F chart data to be used for model training datasets. 
Columns '50 Day Moving Average of Volume' and 'Ratio of Volume to The 50 Day Moving Average' are generated (Does not apply to first 50 datapoints). Categorical columns are refactored to be nomial integers.

In [26]:
def format_data(pf_chart, pred_window, csv, current_ticker):
    # Import security data as a dataframe
    pf_chart_data = pd.DataFrame.from_dict(pf_chart.chart_meta_data)
    pf_chart_data = pf_chart_data.transpose().reset_index(drop=True)


    # Create a 50 day moving average of the volume and a ratio between the volume and the 50 day moving average
    pf_chart_data['50ma'] = pd.Series.rolling(pf_chart_data['volume'],50, min_periods=50).mean().round()
    pf_chart_data['volume_ratio'] = (pf_chart_data['volume'] / pf_chart_data['50ma'])
    
    # Find the change and direction in closing price using the predicted window
    pf_chart_data['shifted_close'] = pf_chart_data['close'].shift(periods=-pred_window)  
    pf_chart_data['window_delta'] =  pf_chart_data['shifted_close'] - pf_chart_data['close']
    pf_chart_data.drop(columns=['shifted_close'],inplace=True)
    pf_chart_data['window_direction'] = pf_chart_data['window_delta'].apply(get_sign)

    # Reformat categorical columns to be numeric (cant decide if these should be numbered by index or polarity ie: buy is 1, sell is -1, none is 0)
    pf_chart_data['action'] = pf_chart_data['action'].map({'none': 0, 'x': 1, 'reverse x->o': 2, 'o': 3, 'reverse o->x': 4})
    pf_chart_data['direction'] = pf_chart_data['direction'].map({'o': 0, 'x':1})
    pf_chart_data['signal'] = pf_chart_data['signal'].map({'sell': 0, 'buy':1}) # We may want to refactor this so none = NaN
    
    # Clean dataset of rows that contain 'nan' and reset the index
    pf_chart_data.dropna(inplace = True, axis = 'rows')
    pf_chart_data.reset_index(inplace=True, drop=True)

    if csv:
        pf_chart_data.to_csv("chart_data-csv/{0}.csv".format(current_ticker)) 
    return pf_chart_data

# Test Classification and Regression With Output

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz

def measure_performance(X,y,clf_gini, output):
    y_pred=clf_gini.predict(X)   
    accuracy = metrics.accuracy_score(y,y_pred)
    classification_report = metrics.classification_report(y,y_pred)
    confusion_matrix = metrics.confusion_matrix(y,y_pred)
    if output:
        print ("Accuracy:{0:.3f}".format(accuracy),"\n")
        print ("Classification report")
        print (classification_report,"\n")
        print ("Confusion matrix")
        print (confusion_matrix,"\n")
    return accuracy, classification_report, confusion_matrix

def CART(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf, output = False, export = False, report = False):
    #define training and test samples
    X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = test_size, random_state = random_state)
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_test = X_test.astype(int)
    y_test = y_test.astype(int)

    #predict using Gini results
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = random_state,
                                   max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    clf_gini.fit(X_train, y_train)
    y_pred = clf_gini.predict(X_test)

    #redefine arrays as integer (otherwise error message results from accuracy calculation)
    y_test = y_test.astype(int)
    y_pred = y_pred.astype(int)
    
    if report:
        accuracy, classification_report, confusion_matrix = measure_performance(X_train,y_train,clf_gini, output)
        report_entry = {
            'ticker': '',
            'test_size': test_size,
            'random_state': random_state, 
            'max_depth': max_depth,
            'min_samples_leaf': min_samples_leaf,
            'accuracy': accuracy,
            'classification_report': classification_report, 
            'confusion_matrix': confusion_matrix,
            'model': clf_gini,
            'X_test': X_test,
            'y_pred': y_pred
        }
        return report_entry
    else:
        measure_performance(X_train,y_train,clf_gini, output)
        
    if output: 
        print ("Values for factors for Gini -",title,":","\n")
    
    if export:
        feature_names=df_names
        with open("cart-models/"+title+"_"+str(test_size)+"_"+str(random_state)+"_"+str(max_depth)+"_"+str(min_samples_leaf)+".txt", "w") as f:
            f = tree.export_graphviz(clf_gini, feature_names=df_names, out_file=f)


# Test Classification and Regression (Dynamic Parameters)

In [23]:
# from pnf import generate_pf_chart
from IPython.display import clear_output
import itertools

def test_models(ticker_set, test_size_set, random_state_set, max_depth_set, min_samples_leaf_set, input_cols, output_col, duration, pred_window, csv = False):
    accuracy_report = pd.DataFrame(columns = ['ticker', 'test_size', 'random_state', 'max_depth', 'min_samples_leaf', 'accuracy', 'classification_report', 'confusion_matrix', 'model', 'X_test', 'y_pred'])
    current_ticker = ''
    i = 1
    
    # Test all combinations of the given objects
    for ticker, test_size, random_state, max_depth, min_samples_leaf in itertools.product(ticker_set, test_size_set, random_state_set, max_depth_set, min_samples_leaf_set):
        if current_ticker != ticker:
            current_ticker = ticker
            clear_output()
            print ('On ticker number {0}/{1}: {2}'.format(str(i),str(len(ticker_set)),ticker))
            i += 1
        
        # Title of the chart
        title = 'CART of ' + ticker 

        # Create a p&f chart for the given ticker using historical data and print the result
        pf_chart = generate_pf_chart(ticker, historical_dfs, duration)
        # Format P&F Chart Data For Analysis
        df = format_data(pf_chart, pred_window, csv, current_ticker)
        #Set the range of the test ()
        X = df.values[:len(df)-pred_window,(input_cols)]
        Y = df.values[pred_window:len(df),output_col]


        # Run the CART model generator
        df_names = list(df.columns.values[input_cols])
        report_entry = CART(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf,report=True)
        report_entry['ticker'] = ticker
        accuracy_report = accuracy_report.append(report_entry, ignore_index=True)

    # Reorder accuracy report descending by accuracy
    accuracy_report = accuracy_report.sort_values(by='accuracy',ascending=False).reset_index(drop=True)
    clear_output()
    print('Done!')
    return accuracy_report

def get_sign(value):
    if value > 0:
        return 1
    else:
        return 0
        

In [27]:
historical_dfs = get_hist_data(universe_tickers)

# Set the tickers and parameter values to be tested 
ticker_set = ['AAPL', 'MMM', 'MMM', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM', 'GS', 'HD' , 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'UTX', 'UNH', 'VZ', 'V', 'WMT', 'WBA']

duration = 5 # Number of years to graph 
pred_window = 5

# Set model parameters to test
test_size_set = [.20,.25,.30] # Percent of the data that is used as training data (Range = 0:1) 
random_state_set = [0,100] # Percent of the data that is randomized in the training/testing data (Range = 0:100)
max_depth_set = [3,4,5] # Max number of layers in the model (Range = 1:inf)
min_samples_leaf_set = [5,15,30] # Minimum number of samples in a group to split on (Range = 1:inf)

# Set input variables to test
input_cols = [1,5,6,8,9,10,15,16,17] # Numeric values or column names of independent varibles
output_col = 19 # Numeric value or column name of the dependent variable

# Create models for each combination of variables
accuracy_report = test_models(ticker_set, test_size_set, random_state_set, max_depth_set, min_samples_leaf_set, input_cols, output_col, duration, pred_window, csv = True)
accuracy_report

Done!


,ticker,test_size,random_state,max_depth,min_samples_leaf,accuracy,classification_report,confusion_matrix,model,X_test,y_pred
0,JNJ,0.30,0,5,15,0.730673,precision recall f1-score s...,"[[139, 176], [40, 447]]","DecisionTreeClassifier(class_weight=None, crit...","[[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...","[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, ..."
1,AAPL,0.30,100,5,5,0.723150,precision recall f1-score s...,"[[216, 175], [57, 390]]","DecisionTreeClassifier(class_weight=None, crit...","[[70, 70, 68, 69, 71, 68, 69829200, 91455070, ...","[1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, ..."
2,JNJ,0.20,0,5,5,0.720829,precision recall f1-score s...,"[[156, 207], [49, 505]]","DecisionTreeClassifier(class_weight=None, crit...","[[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...","[1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, ..."
3,JNJ,0.20,0,5,15,0.717557,precision recall f1-score s...,"[[143, 220], [39, 515]]","DecisionTreeClassifier(class_weight=None, crit...","[[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...","[1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, ..."
4,AAPL,0.30,100,5,15,0.717184,precision recall f1-score s...,"[[232, 159], [78, 369]]","DecisionTreeClassifier(class_weight=None, crit...","[[70, 70, 68, 69, 71, 68, 69829200, 91455070, ...","[1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, ..."
5,MMM,0.20,0,5,5,0.717119,precision recall f1-score s...,"[[219, 175], [96, 468]]","DecisionTreeClassifier(class_weight=None, crit...","[[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...","[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, ..."
6,MMM,0.20,0,5,5,0.717119,precision recall f1-score s...,"[[219, 175], [96, 468]]","DecisionTreeClassifier(class_weight=None, crit...","[[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...","[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, ..."
7,JNJ,0.30,0,5,30,0.716958,precision recall f1-score s...,"[[144, 171], [56, 431]]","DecisionTreeClassifier(class_weight=None, crit...","[[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...","[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, ..."
8,MRK,0.25,0,5,5,0.716078,precision recall f1-score s...,"[[267, 127], [122, 361]]","DecisionTreeClassifier(class_weight=None, crit...","[[52, 52, 51, 51, 52, 50, 12996846, 11593025, ...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, ..."
9,WMT,0.25,100,5,5,0.715420,precision recall f1-score s...,"[[305, 101], [150, 326]]","DecisionTreeClassifier(class_weight=None, crit...","[[65, 67, 65, 67, 69, 64, 12943600, 6616798, 1...","[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, ..."


In [19]:
# Take the best performing model paramters for each stock
ticker_report = accuracy_report.drop_duplicates(subset = 'ticker')

for index, row in ticker_report.iterrows():
    ticker = row['ticker']
    test_size = row['test_size']
    max_depth = row['max_depth']
    random_state = row['random_state']
    min_samples_leaf = row['min_samples_leaf']
    min_samples_leaf = row['min_samples_leaf']
    
    # Recreate the analysis DataFrame using the best performer
    pf_chart = generate_pf_chart(ticker, historical_dfs, duration)
    df = format_data(pf_chart, pred_window)

    X = df.values[:len(df)-1,(input_cols)]
    Y = df.values[1:len(df),output_col]
 
    # Run the CART model generator
    title = 'PnF of ' + ticker 
    df_names = list(df.columns.values[input_cols])

    CART(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf, export = True) 
ticker_report.reset_index(drop=True)

,ticker,test_size,random_state,max_depth,min_samples_leaf,accuracy,classification_report,confusion_matrix,model,X_test,y_pred
0,JNJ,0.30,0,5,15,0.730673,precision recall f1-score s...,"[[139, 176], [40, 447]]","DecisionTreeClassifier(class_weight=None, crit...","[[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...","[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, ..."
1,AAPL,0.30,100,5,5,0.723150,precision recall f1-score s...,"[[216, 175], [57, 390]]","DecisionTreeClassifier(class_weight=None, crit...","[[70, 70, 68, 69, 71, 68, 69829200, 91455070, ...","[1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, ..."
2,MMM,0.20,0,5,5,0.717119,precision recall f1-score s...,"[[219, 175], [96, 468]]","DecisionTreeClassifier(class_weight=None, crit...","[[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...","[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, ..."
3,MRK,0.25,0,5,5,0.716078,precision recall f1-score s...,"[[267, 127], [122, 361]]","DecisionTreeClassifier(class_weight=None, crit...","[[52, 52, 51, 51, 52, 50, 12996846, 11593025, ...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, ..."
4,WMT,0.25,100,5,5,0.715420,precision recall f1-score s...,"[[305, 101], [150, 326]]","DecisionTreeClassifier(class_weight=None, crit...","[[65, 67, 65, 67, 69, 64, 12943600, 6616798, 1...","[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, ..."
5,CVX,0.30,0,5,5,0.712411,precision recall f1-score s...,"[[195, 201], [40, 402]]","DecisionTreeClassifier(class_weight=None, crit...","[[94, 94, 93, 94, 94, 90, 6817000, 6149848, 1]...","[1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, ..."
6,INTC,0.25,100,5,15,0.707127,precision recall f1-score s...,"[[279, 118], [145, 356]]","DecisionTreeClassifier(class_weight=None, crit...","[[21, 21, 21, 21, 21, 20, 31502900, 32757854, ...","[1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, ..."
7,UTX,0.25,100,5,5,0.703786,precision recall f1-score s...,"[[242, 164], [102, 390]]","DecisionTreeClassifier(class_weight=None, crit...","[[100, 100, 98, 99, 100, 97, 2950900, 3016758,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, ..."
8,VZ,0.30,100,5,5,0.695704,precision recall f1-score s...,"[[229, 139], [116, 354]]","DecisionTreeClassifier(class_weight=None, crit...","[[43, 43, 42, 43, 44, 39, 16392100, 15808656, ...","[1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, ..."
9,KO,0.30,0,5,5,0.695704,precision recall f1-score s...,"[[200, 186], [69, 383]]","DecisionTreeClassifier(class_weight=None, crit...","[[33, 33, 32, 33, 33, 31, 16357900, 13266530, ...","[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ..."


Questions:
When dealing with non daily data, what is our measure of 'accuracy'


# Test 6 ML models to compare accuracies
Using the pf_chart_data DataFrame's independent variables, different non-linear functions can be quickly tested to gauge general accuracy in predicting the dependent variable, buy_sell_col. The box and whisker plots provided in the output graph the different results as well as their accuracy spreads.

In [ ]:
# Compare Algorithms
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()
# TODO: Add confusion matrices

In [18]:
accuracy_report['X_test']

0       [[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...
1       [[70, 70, 68, 69, 71, 68, 69829200, 91455070, ...
2       [[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...
3       [[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...
4       [[70, 70, 68, 69, 71, 68, 69829200, 91455070, ...
5       [[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...
6       [[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...
7       [[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...
8       [[52, 52, 51, 51, 52, 50, 12996846, 11593025, ...
9       [[65, 67, 65, 67, 69, 64, 12943600, 6616798, 1...
10      [[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...
11      [[85, 86, 85, 85, 81, 78, 2704600, 2934428, 0]...
12      [[118, 118, 116, 116, 111, 107, 2234300, 24457...
13      [[118, 118, 116, 116, 111, 107, 2234300, 24457...
14      [[78, 79, 77, 77, 78, 74, 11966500, 10119132, ...
15      [[118, 118, 116, 116, 111, 107, 2234300, 24457...
16      [[118, 118, 116, 116, 111, 107, 2234300, 24457...
17      [[94, 